In [57]:
import re
import pandas as pd
import os
import numpy as np

import rapidfuzz
from rapidfuzz import process, fuzz
import difflib
import spacy

In [59]:
path = r'C:\Users\markd\hamilton-county-homes-scraper-main\data\raw\home_sales'
os.chdir(path)
data = pd.read_csv('homes_2011.csv')

path = r'C:\Users\markd\hamilton-county-homes-scraper-main\data\raw'
os.chdir(path)
center = pd.read_csv('Countywide_Street_Centerlines.csv')

C:\Users\markd\AppData\Local\Temp\ipykernel_13160\2187770573.py:7: DtypeWarning: Columns (55,56,57,70,71) have mixed types. Specify dtype option on import or set low_memory=False.
  center = pd.read_csv('Countywide_Street_Centerlines.csv')


In [61]:
cols = ["formatted_address"
                            ,"longitude"
                            ,"latitude"
                            ,"house_num"
                            ,"street_name"
                            ,"api_city"
                            ,"county"
                            ,"api_state"
                            ,"api_postal_code"
                            ,"confidence"]
data.loc[:,cols] = None

In [62]:
data

,parcel_number,address,bbb,finsqft,use,year_built,transfer_date,amount,total_rooms,bedrooms,...,formatted_address,longitude,latitude,house_num,street_name,api_city,county,api_state,api_postal_code,confidence
0,001-0004-0255-00,6355 24 CORBLY RD,4 - 2 - 1 - 1,930,550,1984,7/8/2011,"$105,000",4,2,...,None,None,None,None,None,None,None,None,None,None
1,002-0002-0208-00,6129 CAMBRIDGE AVE,7 - 4 - 1 - 0,1152,510,1927,7/14/2011,"$141,500",7,4,...,None,None,None,None,None,None,None,None,None,None
2,002-0002-0259-00,1602 BRANDON AVE,5 - 2 - 1 - 0,1116,510,1939,4/26/2011,"$114,000",5,2,...,None,None,None,None,None,None,None,None,None,None
3,002-0003-0070-00,1626 ALCOR TE,6 - 3 - 1 - 0,1056,510,1941,7/13/2011,"$127,000",6,3,...,None,None,None,None,None,None,None,None,None,None
4,002-0004-0124-00,1609 CLIO AVE,5 - 2 - 1 - 0,922,510,1939,8/26/2011,"$109,360",5,2,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519,671-0027-0118-00,1304 THURNRIDGE DR,5 - 3 - 2 - 0,1167,510,1954,5/26/2011,"$127,500",5,3,...,None,None,None,None,None,None,None,None,None,None
520,671-0029-0015-00,281 GENOMA DR,5 - 3 - 1 - 0,1134,510,1957,7/29/2011,"$129,900",5,3,...,None,None,None,None,None,None,None,None,None,None
521,671-0030-0008-00,124 WAXWING DR,6 - 3 - 2 - 0,1026,510,1958,5/25/2011,"$151,500",6,3,...,None,None,None,None,None,None,None,None,None,None
522,671-0030-0048-00,2572 S KATHWOOD CR,5 - 3 - 2 - 0,1026,510,1958,9/12/2011,"$129,500",5,3,...,None,None,None,None,None,None,None,None,None,None


In [36]:
center.loc[center["ZIPL"]==' ','ZIPL'].replace(' ',np.nan)

C:\Users\markd\AppData\Local\Temp\ipykernel_13160\3969516827.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  center.loc[center["ZIPL"]==' ','ZIPL'].replace(' ',np.nan)


135     NaN
988     NaN
989     NaN
990     NaN
1837    NaN
1921    NaN
2357    NaN
2760    NaN
4790    NaN
4994    NaN
5475    NaN
5841    NaN
5908    NaN
5952    NaN
5953    NaN
5968    NaN
6000    NaN
6054    NaN
6274    NaN
6358    NaN
6490    NaN
6765    NaN
6783    NaN
6831    NaN
6910    NaN
7152    NaN
7408    NaN
7547    NaN
7871    NaN
7989    NaN
8305    NaN
18854   NaN
18889   NaN
19035   NaN
19905   NaN
20141   NaN
20698   NaN
21413   NaN
29459   NaN
29640   NaN
29758   NaN
33428   NaN
Name: ZIPL, dtype: float64

In [45]:
import pandas as pd
import numpy as np
import os

path = r'C:\Users\markd\hamilton-county-homes-scraper-main\data\raw'
os.chdir(path)

# 1. Load both CSVs
center = pd.read_csv("Countywide_Street_Centerlines.csv", low_memory=False)
zip_df = pd.read_csv("Countywide_Zip_Codes.csv", low_memory=False)

# 2. Prepare the ZIP→city mapping (deduped)
zip_map = (
    zip_df[["ZIPCODE", "USPSCITY"]]
    .dropna(subset=["ZIPCODE", "USPSCITY"])
    .drop_duplicates("ZIPCODE")
    .rename(columns={"ZIPCODE": "zip5", "USPSCITY": "city"})
)
center.loc[center["ZIPL"]==' ','ZIPL'] = center.loc[center["ZIPL"]==' ','ZIPL'].replace(' ',0)
center.loc[center["ZIPR"]==' ','ZIPR'] = center.loc[center["ZIPR"]==' ','ZIPR'].replace(' ',0)


# 3. In centerlines, pick one ZIP per segment
center["zip5"] = center["ZIPL"].fillna(center["ZIPR"]).astype(int)

# 4. Merge (no duplicates in centerlines)
merged = center.merge(zip_map, on="zip5", how="left")

# 5. Inspect
print(merged[["STRLABEL", "L_F_ADD", "R_F_ADD", "zip5", "city"]].head())

C:\Users\markd\AppData\Local\Temp\ipykernel_13160\2547119875.py:19: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  center.loc[center["ZIPL"]==' ','ZIPL'] = center.loc[center["ZIPL"]==' ','ZIPL'].replace(' ',0)
C:\Users\markd\AppData\Local\Temp\ipykernel_13160\2547119875.py:20: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  center.loc[center["ZIPR"]==' ','ZIPR'] = center.loc[center["ZIPR"]==' ','ZIPR'].replace(' ',0)


ValueError: cannot convert float NaN to integer

In [55]:
center["ZIPL"].fillna("ZIPR").fillna(0)

0        45216
1        45202
2        45202
3        45242
4        45202
         ...  
34062    45244
34063    45244
34064    45255
34065    45230
34066    45211
Name: ZIPL, Length: 34067, dtype: object

In [42]:
merged.city.value_counts()

city
Cincinnati       29315
Mason             1660
Loveland           863
Harrison           848
Cleves             534
North Bend         182
Terrace Park       144
Fairfield           84
Hooven              61
Camp Dennison       58
Addyston            47
Lebanon             41
West Chester        18
Hamilton            14
Milford             13
Maineville           3
Middletown           3
Covington            2
Name: count, dtype: int64

In [3]:
direction_map = {
    "N": "NORTH",
    "S": "SOUTH",
    "E": "EAST",
    "W": "WEST",
    "NW": "NORTHWEST",
    "SW": "SOUTHWEST",
    "NE": "NORTHEAST",
    "SE": "SOUTHEAST",
    "NB": "NORTHBOUND",
    "SB": "SOUTHBOUND",
    "EB":"EASTBOUND",
    "WB":"WESTBOUND",
    }

In [5]:
street_type_map = {
    # Avenue
    'AVE': 'AVENUE',
    'AV': 'AVENUE',
    'AVEN': 'AVENUE',
    'AVENU': 'AVENUE',

    # Drive
    'DR': 'DRIVE',
    'DRV': 'DRIVE',

    # Road
    'RD': 'ROAD',
    'ROAD': 'ROAD',

    # Court
    'CT': 'COURT',
    'CRT': 'COURT',

    # Lane
    'LN': 'LANE',
    'LAN': 'LANE',
    'LA': 'LANE',

    # Street
    'ST': 'STREET',
    'STR': 'STREET',

    # Circle
    'CR': 'CIRCLE',
    'CIR': 'CIRCLE',
    'CRCLE': 'CIRCLE',

    # Place
    'PL': 'PLACE',
    'PLC': 'PLACE',

    # Way
    'WY': 'WAY',
    'WAY': 'WAY',

    # Terrace
    'TERR': 'TERRACE',
    'TER': 'TERRACE',
    'TE': 'TERRACE',
    'TCE': 'TERRACE',

    # Pike
    'PK': 'PIKE',
    'PIKE': 'PIKE',
    'PKE': 'PIKE',

    # Square
    'SQ': 'SQUARE',
    'SQR': 'SQUARE',
    'SQRE': 'SQUARE',

    # Boulevard
    'BLVD': 'BOULEVARD',
    'BV': 'BOULEVARD',
    'BLV': 'BOULEVARD',

    # Trail
    'TRL': 'TRAIL',
    'TL': 'TRAIL',
    'TRAIL': 'TRAIL',

    # Parkway
    'PKWY': 'PARKWAY',
    'PKY': 'PARKWAY',
    'PW': 'PARKWAY',
    'PKWAY': 'PARKWAY',

    # Trace
    'TRCE': 'TRACE',
    'TR': 'TRACE',
    'TRACE': 'TRACE',

    # Additional common types:
    'XING': 'CROSSING',
    'CRSG': 'CROSSING',
    'EXPY': 'EXPRESSWAY',
    'EXWY': 'EXPRESSWAY',
    'FWY': 'FREEWAY',
    'HTS': 'HEIGHTS',
    'MT': 'MOUNT',
    'RTE': 'ROUTE',
    'VIS': 'VISTA',
    'CRES': 'CRESCENT',
    'ALY': 'ALLEY',
    'BND': 'BEND',
}


In [34]:
# pad maps with spaces so we don't hit substrings
_SUFFIX_MAP = {k: v for k, v in street_type_map.items()}
_DIR_MAP    = {k: v for k, v in direction_map.items()}

In [ ]:
nlp = spacy.load("en_core_web_sm")

def _canon(name: str) -> str:
    """
    Return a fully upper-case, punctuation-stripped, suffix-expanded
    street string suitable for matching/fuzzy lookup.
    """
    n = " " + re.sub(r"[^\w\s]", " ", name.upper()) + " "

    # move trailing direction to the front (e.g. 'GALBRAITH RD NW')
    m = re.search(r"\b(NORTH|SOUTH|EAST|WEST|NORTHWEST|SOUTHWEST|NORTHEAST|SOUTHEAST)$", n)
    if m:
        direction = m.group(0)
        n = direction + " " + n.replace(direction, "").strip()

    # expand directions
    for k, v in _DIR_MAP.items():
        n = n.replace(k, v)

    # expand only the final token if it matches a street type
    tokens = n.split()
    if tokens:
        last = f" {tokens[-1]} "
        if last in _SUFFIX_MAP:
            tokens[-1] = _SUFFIX_MAP[last].strip()
    n = " ".join(tokens)
    return re.sub(r"\s+", " ", n).strip()

def _closest_name(bad, valid_names, score_cut=80):
    cand, score, _ = process.extractOne(
        bad, valid_names, scorer=fuzz.token_set_ratio
    )

    return cand if cand and score >= score_cut else bad

def add_zip_code(df: pd.DataFrame,
                 centerline_path: str = "Countywide_Street_Centerlines.csv",
                 address_col: str = "Address") -> pd.DataFrame:
    """
    Return a copy of df with a new 'ZIP' column.
    Requires df[address_col] to contain full street addresses as scraped.
    """
    # --- load & prepare centerline reference ------------------------
    center = pd.read_csv(centerline_path, low_memory=False)
    for col in ["STRLABEL", "L_F_ADD", "L_T_ADD", "R_F_ADD",
                "R_T_ADD", "ZIPL", "ZIPR"]:
        if col not in center.columns:
            raise ValueError(f"Centerline CSV missing expected column '{col}'")

    center["CANON"] = center["STRLABEL"]
    gold = center["CANON"].unique()

    # group by street for fast range checks
    grouped = center.groupby("CANON")

    df = df.copy()
    df["postal_code"] = df[address_col].apply(_zip_for_row)
    df["address"] = df[address_col].apply(
        lambda a: _closest_name(gold)
    )
    return df

def is_alphanumeric(token):
    """Check if the token text is alphanumeric."""
    return re.match("^(?=.*[0-9])(?=.*[a-zA-Z])[a-zA-Z0-9]+$", token.text) is not None

def tag_address(address: str) -> dict:
    """
    Parse a raw address string into components:
    st_num, apt_num, pre_dir, street_base, street_type, post_dir,
    plus a canonicalised 'street_corrected' you can feed to the ZIP lookup.
    """
    spelled_out_numbers = {
        "zero","one","two","three","four","five","six","seven","eight","nine",
        "ten","eleven","twelve","thirteen","fourteen","fifteen","sixteen",
        "seventeen","eighteen","nineteen","twenty","first","second","third",
        "fourth","fifth","sixth","seventh","eighth","ninth","tenth",
        "eleventh","twelfth","thirteenth","fourteenth","fifteenth",
        "sixteenth","seventeenth","eighteenth","nineteenth"
    }

    doc  = nlp(address.upper())
    toks = [t.text.strip(".") for t in doc if t.text.strip()]

    out = {
        "st_num": None, "apt_num": None,
        "pre_dir": None, "street_base": None,
        "street_type": None, "post_dir": None,
    }

    # ---------- 1. house / apt -----------------
    idx = 0
    if idx < len(toks) and toks[idx].isdigit():
        out["st_num"] = toks[idx]; idx += 1

    if idx < len(toks) and (
        is_alphanumeric(doc[idx]) or doc[idx].pos_ == "NUM"
    ) and toks[idx].lower() not in spelled_out_numbers:
        out["apt_num"] = toks[idx]; idx += 1

    # ---------- 2. prefix direction ------------
    if idx < len(toks) and toks[idx] in direction_map:
        out["pre_dir"] = direction_map[toks[idx]]
        idx += 1

    # ---------- 3. scan remaining tokens -------
    buf = []
    while idx < len(toks):
        tok = toks[idx]
        # treat the *last* directional / type tokens specially
        if idx == len(toks) - 1 and tok in direction_map:
            out["post_dir"] = direction_map[tok]
        elif idx == len(toks) - 1 and tok in street_type_map:
            out["street_type"] = street_type_map[tok]
        else:
            buf.append(tok)
        idx += 1

    out["street_base"] = " ".join(buf).strip()

    # ---------- 4. canonical full street -------
    parts = [
        out["pre_dir"], out["street_base"],
        out["street_type"], out["post_dir"]
    ]
    full = " ".join(p for p in parts if p)
    out["street_corrected"] = full if full else None
    return out

# ------------------------------------------------------------------
#  ADDRESS ENRICHER – one-time loader you can reuse in callbacks etc.
# ------------------------------------------------------------------
class AddressEnricher:
    """
    Parse an address string, fuzzy-correct the street name,
    lookup ZIP from the center-line table, and return a dict of fields.
    """
    def __init__(self, center: pd.DataFrame, score_cut: int = 80):
        needed = ["STRLABEL", "L_F_ADD", "L_T_ADD",
                  "R_F_ADD", "R_T_ADD", "ZIPL", "ZIPR"]
        missing = [c for c in needed if c not in center.columns]
        if missing:
            raise ValueError(f"Centerline CSV missing {missing}")

        center = center.copy()
        center["CANON"] = center["STRLABEL"].apply(_canon)

        self._grouped   = center.groupby("CANON")
        self._gold      = center["CANON"].unique()
        self._score_cut = score_cut

    # ------------------------------------------------------------------
    def _zip_from_ranges(self, canon_street, hnum):
        segs = self._grouped.get_group(canon_street)

        seg = segs.loc[
            ((segs.L_F_ADD <= hnum) & (hnum <= segs.L_T_ADD)) |
            ((segs.R_F_ADD <= hnum) & (hnum <= segs.R_T_ADD))
        ]
        if seg.empty:
            return None
        seg = seg.iloc[0]

        if hnum % 2 == 0:        # even house number
            return seg.ZIPL if seg.L_F_ADD % 2 == 0 else seg.ZIPR
        else:                    # odd
            return seg.ZIPR if seg.R_F_ADD % 2 == 1 else seg.ZIPL

    # ------------------------------------------------------------------
    def enrich(self, raw_address: str) -> dict:
        tags = tag_address(raw_address)  # returns st_num / street / etc.

        if not tags["street_corrected"]:
            return {**tags, "postal_code": None, "street_corrected": None}

        canon_street = tag_address(tags["street_corrected"])
        if canon_street not in self._grouped.groups:
            canon_street = _closest_name(
                canon_street, self._gold, score_cut=self._score_cut
            )

        zip_code = None
        hnum = int(tags["st_num"]) if tags["st_num"] else None
        if canon_street and hnum is not None and canon_street in self._grouped.groups:
            zip_code = self._zip_from_ranges(canon_street, hnum)

        return {**tags,
                "postal_code": zip_code,
                "street_corrected": canon_street}



# instantiate once so every downstream module can import & reuse it
address_enricher = AddressEnricher(center)

In [43]:
tags = tag_address('2331 Grant Ave')

In [41]:
tags

{'st_num': '2331',
 'apt_num': None,
 'pre_dir': None,
 'street_base': 'GRANT',
 'street_type': 'AVENUE',
 'post_dir': None,
 'street_corrected': 'GRANT AVENUE'}

In [46]:
print(address_enricher.enrich('2331 Grant Ave'))

TypeError: unhashable type: 'dict'

In [11]:
for i, j in zip(center["FEATUREID"], center["STRLABEL"]):
    result = tag_address(j)
    center.loc[center["FEATUREID"]==i, "STREET_NORM"] = result['street_corrected']

In [12]:
center

,Unnamed: 0,OBJECTID,ENABLED,FEATUREID,STRSEGID,SEGIDUSNG,REC_OWNER,STRNAMID,STRLABEL,SEGTYPE,...,MAINT_LEFT,MAINT_RIGH,GLOBALID,CREATED_USER,CREATED_DATE,LAST_EDITED_USER,LAST_EDITED_DATE,SHAPELEN,CANON,STREET_NORM
0,0,1,0.0,HAM26799,3820796674538311197596,GJ1857943124GJ1866443388,MUNICIPALITIES,WOODB01,WOODBINE AV,ROAD,...,CPUBSV,CPUBSV,{9AC80B4C-7D3D-46A2-9462-101931D5286F},NaN,NaN,DTE_STEVENSON,2017/04/22 16:03:29+00,0,WOODBINE AV,WOODBINE AVENUE
1,1,2,0.0,HAM08531,3306204270933063722736,GJ1465132539GJ1470232549,MUNICIPALITIES,ELDER04W,W ELDER ST,ROAD,...,CPUBSV,CPUBSV,{84EED39F-2B8E-41D4-9496-75EE35803918},NaN,NaN,DTE_JTRUMAN,2023/04/26 16:36:43+00,0,W ELDER ST,WEST ELDER STREET
2,2,3,0.0,HAM08536,3306066268533062042709,GJ1460932529GJ1465132539,MUNICIPALITIES,ELDER04W,W ELDER ST,ROAD,...,CPUBSV,CPUBSV,{469938FB-C63B-4282-B7B1-7971721F799E},NaN,NaN,DTE_JTRUMAN,2023/04/26 16:36:51+00,0,W ELDER ST,WEST ELDER STREET
3,3,4,0.0,HAM61153,4969652181649696733358,GJ2727245575GJ2728945105,ODOT,I-71 35,I-71 SB EXWY,ROAD,...,,,{A338D1BF-8AAA-49E9-B2AF-1C4CC0F174E7},NaN,NaN,TTURINSKY,2016/07/01 14:29:26+00,0,I-71 SB EXWY,I-71 SB EXPRESSWAY
4,4,5,1.0,HAM52712,3360903154633612883256,GJ1610032256GJ1619132782,ODOT,I-71 34,I-71 NB EXWY,HIGHWAY,...,ODOT,ODOT,{E348F40E-45D8-4BE2-81C1-09D1B686DF5B},NaN,NaN,DTE_JTRUMAN,2018/01/18 16:46:47+00,0,I-71 NB EXWY,I-71 NB EXPRESSWAY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34062,34062,286257,NaN,HAM72956,5735462283357358803010,GJ3269433474GJ3281833534,ANDETP,MEADO51,MEADOW VIEW LN,ROAD,...,NaN,NaN,{2B14A0C7-8CC5-4EEA-B37C-D95D435220B5},HCPW_WBOWLES,2024/08/21 13:44:16+00,HCPW_WBOWLES,2024/08/21 13:44:54+00,0,MEADOW VIEW LN,MEADOW VIEW LANE
34063,34063,286258,0.0,HAM72957,5735462283357355702451,GJ3273333359GJ3269433474,HCE,MT CA01,MT CARMEL RD,ROAD,...,COUNTY,COUNTY,{ED2712AB-C3C8-47FC-A6D9-43FBFC03D76C},HCPW_WBOWLES,2024/08/21 13:46:11+00,HCPW_WBOWLES,2024/08/21 13:47:38+00,0,MT CARMEL RD,MT CARMEL ROAD
34064,34064,286259,0.0,HAM72958,5601129624456011376388,GJ3162528357GJ3162628400,HCE,EIGHT01,EIGHT MILE RD,ROAD,...,COUNTY,COUNTY,{5F195EF6-C31B-49A5-8412-7C164597A94E},HCPW_WBOWLES,2024/08/21 14:20:07+00,HCPW_WBOWLES,2024/08/21 14:22:18+00,0,EIGHT MILE RD,EIGHT MILE ROAD
34065,34065,286545,NaN,HAM72972,4170307372141703223512,GJ2227827122GJ2228627058,CINC,VALLE06,VALLEY FORD ST,ROAD,...,PRIVATE,PRIVATE,{C7A6BC52-B229-46BF-A748-814ECDD793B9},DTE_JTRUMAN,2024/08/30 16:45:50+00,DTE_JTRUMAN,2024/08/30 16:50:02+00,0,VALLEY FORD ST,VALLEY FORD STREET


In [22]:
results_list = []
for i,j in zip(data.parcel_number, data.address):
    result = tag_address(j)
    results_list.append(result)

In [23]:
pd.DataFrame(results_list)

,st_num,apt_num,pre_dir,street_base,street_type,post_dir,street_corrected
0,6329,None,None,COFFEY,STREET,None,COFFEY STREET
1,6520,None,None,RAINBOW,LANE,None,RAINBOW LANE
2,6574,None,None,COFFEY,STREET,None,COFFEY STREET
3,6564,None,None,RAINBOW,LANE,None,RAINBOW LANE
4,6555,None,None,SILVERFOX,DRIVE,None,SILVERFOX DRIVE
...,...,...,...,...,...,...,...
848,1324,None,None,THURNRIDGE,DRIVE,None,THURNRIDGE DRIVE
849,1167,None,None,ALWIL,DRIVE,None,ALWIL DRIVE
850,1321,None,None,FUHRMAN,ROAD,None,FUHRMAN ROAD
851,1147,None,None,THURNRIDGE,DRIVE,None,THURNRIDGE DRIVE
